In [296]:
import numpy as np
import torch.nn as nn
import time

In [297]:
def gen_weight(numFilters, Df,Hf,Wf):
    ret = np.zeros((numFilters, Df,Hf,Wf))
    for f in range(0, numFilters):
        for d in range(0, Df):
            for i in range(0, Hf):
                for j in range(0, Wf):
                    test = np.random.uniform()
                    if test < 1/3:
                        ret[f,d,i,j] = -1
                    if test > 2/3:
                        ret[f,d,i,j] = 1
    return ret

In [298]:
#first of all, divide weights by groups
#we need a starting segment and an init segment
#everything will be in one loop?

In [299]:
def gen_conv_c_code(weight, n_groups, Hin, Win, stride):
    filter_depth = weight.shape[1]
    Dout = weight.shape[0]
    kernel_size = weight.shape[3]
    Hout = (Hin - kernel_size)//stride + 1
    Wout = (Win - kernel_size)//stride + 1
    filters_per_group = Dout//n_groups
    Din = filter_depth*n_groups
    how_many_steps_to_do = Hout*Wout
    groups = []
    cur = 0
    for i in range(0, n_groups):
        new_group = []
        for j in range(cur, cur+filters_per_group):
            new_group.append(weight[j])
        cur += filters_per_group
        groups.append(new_group)
    output = f''
    output += f'#include <stdio.h>\n'
    output += f'#include <time.h>\n'
    output += f'#define INARRLEN {Din*Hin*Win}\n'
    output += f'#define OUTARRLEN {Dout*Hout*Wout}\n'
    output += f'float I[INARRLEN];\n'
    output += f'float O[OUTARRLEN];\n'
    output += f'int main(){{\n'
    output += f'FILE* input_file = fopen("input.txt", "r");\n'
    output += f'for (long int i = 0; i < INARRLEN; i++){{fscanf(input_file, "%f", &I[i]);}}\n'
    output += f'fclose(input_file);\n'
    output += f'clock_t st, end;\n'
    output += f'st = clock();\n'
    starting_position_of_group = 0
    fno = 0
    for g in range(0, n_groups):
        for f in range(0, filters_per_group):
            for d in range(0, filter_depth):
                for h in range(0, kernel_size):
                    output += f'int i_{fno}_{d}_{h} = {starting_position_of_group + d*Hin*Win + h*Win};\n'
            fno += 1
        starting_position_of_group += filter_depth*Hin*Win
    output_counter = 0
    for i in range(0, Dout):
        output += f'int o_{i} = {output_counter};\n'
        output_counter += Hout*Wout
    output += f'for(long int i = 0; i < {0 + how_many_steps_to_do}; i += 1){{\n'
    for f in range(0, Dout):
        
        for d in range(0, filter_depth):
            for h in range(0, kernel_size):
                output += f'\tO[o_{f}]+='
                for w in range(0, kernel_size):
                    if weight[f, d, h, w] != 0:
                        if weight[f, d, h, w] == 1:
                            output += f'+I[i_{f}_{d}_{h}+{w}]'
                        if weight[f, d, h, w] == -1:
                            output += f'-I[i_{f}_{d}_{h}+{w}]'
                output += '+0;\n'
    for f in range(0, Dout):
        for d in range(0, filter_depth):
            for h in range(0, kernel_size):
                output+=f'\ti_{f}_{d}_{h} += {stride};\n'

    amount_to_jump_after_each_row =    -stride + Win - stride*(Wout-1) + (stride-1)*Win#12
    output += f'\tif ((i+1) % {Wout} == 0){{\n'
    for f in range(0, Dout):
        for d in range(0, filter_depth):
            for h in range(0, kernel_size):
                output+=f'\t\ti_{f}_{d}_{h} += {amount_to_jump_after_each_row};\n'
    output += f'\t}}\n'

    for f in range(0, Dout):
        output += f'\to_{f} += 1;\n'
    output += f'}}\n'
    output += f'end = clock();\n'
    output += f'printf("Conv time = %f\\n", ((double)(end - st))/CLOCKS_PER_SEC);\n'
    output += f'FILE* output_file = fopen("output.txt", "w");\n'
    output += f'for (long int i = 0; i < OUTARRLEN; i++){{fprintf(output_file,"%f\\n", O[i]);}}\n'
    output += f'fclose(output_file);\n'
    output += f'return 0;\n}}\n'
    return output





In [300]:
def genOneConvFilterNOIIB(weight,dinin,dinout,inputH,inputW,INPUTARRAYNAME,OUTPUTARRAYNAME,outputH,outputW):
    kernel_depth = weight.shape[0]
    kernel_size = weight.shape[1]
    output = f'{OUTPUTARRAYNAME}[iib({dinout},iout,jout,{outputH},{outputW})]+='
    nonzerow = 0
    for dr in range(0,kernel_depth):
        for ir in range(0, kernel_size):
            for jr in range(0, kernel_size):
                if weight[dr,ir,jr] == 1:
                    output+=f'+{INPUTARRAYNAME}[fs+{dinin*inputH*inputW+dr*inputH*inputW+ir*inputW+jr}]'
                    nonzerow += 1
                if weight[dr,ir,jr] == -1:
                    output+=f'-{INPUTARRAYNAME}[fs+{dinin*inputH*inputW+dr*inputH*inputW+ir*inputW+jr}]'
                    nonzerow += 1
    output+='+0;\n'
    return output

def genBitConvBitPythonNOIIB(weight,
                             in_channels,out_channels,
                             kernel_size,stride,
                             groups,
                             inputH,inputW):

    outputH = (inputH-kernel_size)//stride+1
    outputW = (inputW-kernel_size)//stride+1
    fDefCode =f'''
int i = 0;
int j = 0;
'''
    fDefCode += f'for (int iout = 0; iout < {outputH-0}; iout++){{\n'
    fDefCode+=f'\tfor (int jout = {0}; jout < {outputW-0}; jout++){{\n\t\t'
    fDefCode+=f'long int fs = j+{inputW}*i;'
    filterDepth = in_channels//groups
    filtersPerGroup = out_channels//groups
    dinin = 0
    for dinout in range(0, out_channels):
        if dinout > 0 and dinout%filtersPerGroup==0:
            dinin += filterDepth
        fDefCode+= genOneConvFilterNOIIB(weight[dinout],
                                           dinin,dinout,
                                           inputH,inputW,
                                           'I',
                                           'O',
                                           outputH,outputW)
    fDefCode += f"\n\t\tj+={stride};}}\n\ti+={stride};\n\tj=0;}}\n"
    return fDefCode





def gen_legacy_conv(weight, n_groups, Hin, Win, stride):
    filter_depth = weight.shape[1]
    Dout = weight.shape[0]
    kernel_size = weight.shape[3]
    Hout = (Hin - kernel_size)//stride + 1
    Wout = (Win - kernel_size)//stride + 1
    Din = filter_depth*n_groups
    output = f''
    output += f'#include <stdio.h>\n'
    output += f'#include <time.h>\n'
    output += f'#define INARRLEN {Din*Hin*Win}\n'
    output += f'#define OUTARRLEN {Dout*Hout*Wout}\n'
    output += f'long int iib(int d,int i,int j,int H,int W){{return d*(H*W)+i*W+j;}}\n'
    output += f'float I[INARRLEN];\n'
    output += f'float O[OUTARRLEN];\n'
    output += f'int main(){{\n'
    output += f'FILE* input_file = fopen("input.txt", "r");\n'
    output += f'for (long int i = 0; i < INARRLEN; i++){{fscanf(input_file, "%f", &I[i]);}}\n'
    output += f'fclose(input_file);\n'
    output += f'clock_t st, end;\n'
    output += f'st = clock();\n'

    output += genBitConvBitPythonNOIIB(weight=weight,
                             in_channels=weight.shape[1]*n_groups,
                             out_channels=weight.shape[0],
                             kernel_size=weight.shape[2],
                             stride=stride,
                             groups=n_groups,
                             inputH=Hin,
                             inputW=Win)

    output += f'end = clock();\n'
    output += f'printf("Conv time = %f\\n", ((double)(end - st))/CLOCKS_PER_SEC);\n'
    output += f'FILE* output_file = fopen("output_legacy.txt", "w");\n'
    output += f'for (long int i = 0; i < OUTARRLEN; i++){{fprintf(output_file,"%f\\n", O[i]);}}\n'
    output += f'fclose(output_file);\n'
    output += f'return 0;\n}}\n'
    return output

In [304]:
kernel_size = 3
stride = 1
in_channels = 3
out_channels = 3
groups = 3
padding = 0
Hin = 32
Win = 32
w = gen_weight(out_channels, in_channels//groups, kernel_size, kernel_size)
w_tensor = torch.tensor(w,dtype=torch.float)
inputto = torch.rand((1, in_channels, Hin, Win))
to_write_to_file = torch.squeeze(inputto, 0)
to_write_to_file = to_write_to_file.flatten().detach().numpy()
np.savetxt("input.txt", to_write_to_file)
st = time.time()
outputto = torch.nn.functional.conv2d(inputto, w_tensor,bias=None,stride=stride, padding=padding,dilation=1,groups=groups)
end = time.time()
print("pytorch time = ", end - st)
outputto = torch.squeeze(outputto, 0)
print("OUTPUT SHAPE = ", outputto.shape)
outputto = outputto.flatten()

stroka = gen_conv_c_code(w, groups, Hin, Win, stride)
#print(stroka)
with open('one_conv_code.c', 'w') as f:
    f.write(stroka)

!cd /home/oleksiy-tsuber/gen_conv_cache_local
!gcc -g -O2 -o one_conv_code one_conv_code.c
!./one_conv_code 
#!valgrind --tool=callgrind --dump-instr=yes --collect-jumps=yes --simulate-cache=yes ./one_conv_code

stroka_legacy = gen_legacy_conv(w, groups, Hin, Win, stride)
with open('conv_legacy.c', 'w') as f:
    f.write(stroka_legacy)
!gcc -g -O2 -o conv_legacy conv_legacy.c
!./conv_legacy
#!valgrind --tool=callgrind --dump-instr=yes --collect-jumps=yes --simulate-cache=yes ./conv_legacy

output_mine = torch.tensor(np.loadtxt('output.txt'))
print('new conv acc: ', torch.mean(torch.abs(outputto - output_mine)))

output_legacy = torch.tensor(np.loadtxt('output_legacy.txt'))
print('legacy conv acc: ', torch.mean(torch.abs(outputto - output_legacy)))


pytorch time =  0.034807682037353516
OUTPUT SHAPE =  torch.Size([3, 30, 30])
one_conv_code.c: In function ‘main’:
one_conv_code.c:9:41: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
    9 | for (long int i = 0; i < INARRLEN; i++){fscanf(input_file, "%f", &I[i]);}
      |                                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Conv time = 0.000006
conv_legacy.c: In function ‘main’:
conv_legacy.c:10:41: warning: ignoring return value of ‘fscanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   10 | for (long int i = 0; i < INARRLEN; i++){fscanf(input_file, "%f", &I[i]);}
      |                                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Conv time = 0.000004
new conv acc:  tensor(2.5726e-07, dtype=torch.float64)

In [305]:
print(stroka)

#include <stdio.h>
#include <time.h>
#define INARRLEN 3072
#define OUTARRLEN 2700
float I[INARRLEN];
float O[OUTARRLEN];
int main(){
FILE* input_file = fopen("input.txt", "r");
for (long int i = 0; i < INARRLEN; i++){fscanf(input_file, "%f", &I[i]);}
fclose(input_file);
clock_t st, end;
st = clock();
int i_0_0_0 = 0;
int i_0_0_1 = 32;
int i_0_0_2 = 64;
int i_1_0_0 = 1024;
int i_1_0_1 = 1056;
int i_1_0_2 = 1088;
int i_2_0_0 = 2048;
int i_2_0_1 = 2080;
int i_2_0_2 = 2112;
int o_0 = 0;
int o_1 = 900;
int o_2 = 1800;
for(long int i = 0; i < 900; i += 1){
	O[o_0]+=-I[i_0_0_0+0]-I[i_0_0_0+1]+I[i_0_0_0+2]+0;
	O[o_0]+=+I[i_0_0_1+0]+I[i_0_0_1+1]-I[i_0_0_1+2]+0;
	O[o_0]+=-I[i_0_0_2+1]+I[i_0_0_2+2]+0;
	O[o_1]+=-I[i_1_0_0+0]-I[i_1_0_0+1]+0;
	O[o_1]+=-I[i_1_0_1+0]-I[i_1_0_1+2]+0;
	O[o_1]+=-I[i_1_0_2+1]+0;
	O[o_2]+=+I[i_2_0_0+0]+0;
	O[o_2]+=-I[i_2_0_1+0]-I[i_2_0_1+2]+0;
	O[o_2]+=-I[i_2_0_2+1]+0;
	i_0_0_0 += 1;
	i_0_0_1 += 1;
	i_0_0_2 += 1;
	i_1_0_0 += 1;
	i_1_0_1 += 1;
	i_1_0_2 += 1;
	i_2_0_0 += 1;


In [306]:
print(stroka_legacy)

#include <stdio.h>
#include <time.h>
#define INARRLEN 3072
#define OUTARRLEN 2700
long int iib(int d,int i,int j,int H,int W){return d*(H*W)+i*W+j;}
float I[INARRLEN];
float O[OUTARRLEN];
int main(){
FILE* input_file = fopen("input.txt", "r");
for (long int i = 0; i < INARRLEN; i++){fscanf(input_file, "%f", &I[i]);}
fclose(input_file);
clock_t st, end;
st = clock();

int i = 0;
int j = 0;
for (int iout = 0; iout < 30; iout++){
	for (int jout = 0; jout < 30; jout++){
		long int fs = j+32*i;O[iib(0,iout,jout,30,30)]+=-I[fs+0]-I[fs+1]+I[fs+2]+I[fs+32]+I[fs+33]-I[fs+34]-I[fs+65]+I[fs+66]+0;
O[iib(1,iout,jout,30,30)]+=-I[fs+1024]-I[fs+1025]-I[fs+1056]-I[fs+1058]-I[fs+1089]+0;
O[iib(2,iout,jout,30,30)]+=+I[fs+2048]-I[fs+2080]-I[fs+2082]-I[fs+2113]+0;

		j+=1;}
	i+=1;
	j=0;}
end = clock();
printf("Conv time = %f\n", ((double)(end - st))/CLOCKS_PER_SEC);
FILE* output_file = fopen("output_legacy.txt", "w");
for (long int i = 0; i < OUTARRLEN; i++){fprintf(output_file,"%f\n", O[i]);}
fclose(outp